In [ ]:
import os
import torch
import matplotlib.pyplot as plt

In [ ]:
# must be run only once per session
os.chdir("..")

In [ ]:
from detectors import YoloV8
from utils import YamlParser, TrackingRunner

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device, type(device)

## 1. Initialize object detector (YoloV8)

In [ ]:
detector = YoloV8(
    weights="yolov8n.pt",
    conf_thres=0.45,
    display_labels=True,
    device=device
)

### 1.1. (Optional) Only detect specific classes

In [ ]:
#### view class mapping
detector.class_names_dict

In [ ]:
# (Optional) Only execute below line if want to detect only specific classes

# e.g. detect only bicycles and cars
detector.detect_classes(classes=[1, 2])

### 1.2. (Optional) Only detect in specific zone/ region

In [ ]:
# (Optional) Only execute below lines if want to detect only in specific zones

# define polygon zones (from start_pixel_location to end_pixel_location)
zone1 = [
    [145, 120],
    [10,  295],
    [360, 295],
    [348, 203],
    [320, 120]
]

zone2 = [
    [506, 200],
    [462, 238],
    [390, 340],
    [542, 340],
    [588, 238],
    [618, 200]
]

zones = [zone1, zone2]

detector.detect_zones(
    zones=zones,
    frame_resolution_wh=(640, 360),
)

# ----------------------------------------------------------------------------

## 2. Initialize tracker
#### ------------------------------------------

- There are various types of Trackers for performing Multi Object Tracking (MOT).
- SORT - Simple Online & Realtime Tracking (2 Feb 2016; (https://arxiv.org/abs/1602.00763) was a breakthrough in the field of trackers in the year 2016. It gained a lot of attention and eventually many other trackers are implemented based on its foundation.
- Then came the DeepSORT (21 Mar 2017) build on top of SORT which uses deep learning for target associations and is widely used even today.
- Thus, there are many trackers that are implemented till now and currently, the new **State-of-the-Art** (SOTA) trackers are **OC-SORT, ByteTrack, BotSORT, StrongSORT, Deep OC-SORT, ** which are released in the year **2022-2023**.

### Creating a custom feature extraction model for ReIDentification task

- ReID model is used for extracting the features of the object (detected by object detectors like Yolov8)
- DeepSORT, BoTSORT, StrongSORT and DeepOCSORT.
- These trackers use the ReID model. Hence, these are deep learning based MOT trackers and are used when performing inference on GPUs.
- These trackers are normally not used for CPU inferencing.

- You can train a resnet18 or resnet50 model on a custom dataset and can provide the model weight path to the respective tracking class (you can find this in later cells of this notebook for the above mentioned trackers)

#### -----------------------------
- Please note, the dataset you are supposed to train the ReID model should contain only the objects in it. i.e. the images should be cropped so that the cropped images only contain the object.
- Create a dataset of such cropped images for each class

### 2.1. SORT [2 Feb 2016]
#### Simple Online and Realtime Tracking
#### https://arxiv.org/abs/1602.00763

In [ ]:
from trackers import SORT

In [ ]:
tracker_config = os.path.join(os.getcwd(), "trackers", "sort", "configs", "sort.yaml")

cfg = YamlParser()
cfg.merge_from_file(tracker_config)

tracker = SORT(
    max_age=cfg.sort.max_age,
    min_hits=cfg.sort.min_hits,
    iou_threshold=cfg.sort.iou_threshold
)

### 2.2. DeepSORT [21 Mar 2017]
#### Simple Online and Realtime Tracking with a Deep Association Metric
#### https://arxiv.org/abs/1703.07402

In [ ]:
from trackers import DeepSORT

In [ ]:
reid_weights = os.path.join(os.getcwd(), "trackers", "deepsort", "deep", "checkpoint", "osnet_x0_25_msmt17.pt")
tracker_config = os.path.join(os.getcwd(), "trackers", "deepsort", "configs", "deepsort.yaml")

cfg = YamlParser()
cfg.merge_from_file(tracker_config)

tracker = DeepSORT(
    model_weights=reid_weights,
    device=device,
    fp16=True,
    max_dist=cfg.deepsort.max_dist, 
    min_confidence=cfg.deepsort.min_confidence, 
    nms_max_overlap=cfg.deepsort.nms_max_overlap, 
    max_iou_distance=cfg.deepsort.max_iou_distance, 
    max_age=cfg.deepsort.max_age, 
    n_init=cfg.deepsort.n_init, 
    nn_budget=cfg.deepsort.nn_budget
)

### 2.3. StrongSORT [28 Feb 2022]
#### Make DeepSORT Great Again  (Catchy Slogan! It is based on a popular tracker: DeepSORT)
#### https://arxiv.org/abs/2202.13514

In [ ]:
from trackers import StrongSORT

In [ ]:
reid_weights = os.path.join(os.getcwd(), "trackers", "strongsort", "deep", "checkpoint", "osnet_x0_25_msmt17.pt")
tracker_config = os.path.join(os.getcwd(), "trackers", "strongsort", "configs", "strongsort.yaml")

cfg = YamlParser()
cfg.merge_from_file(tracker_config)

tracker = StrongSORT(
    model_weights=reid_weights,
    device=device,
    fp16=True,
    max_dist=cfg.strongsort.max_dist,
    max_iou_dist=cfg.strongsort.max_iou_dist,
    max_age=cfg.strongsort.max_age,
    max_unmatched_preds=cfg.strongsort.max_unmatched_preds,
    n_init=cfg.strongsort.n_init,
    nn_budget=cfg.strongsort.nn_budget,
    mc_lambda=cfg.strongsort.mc_lambda,
    ema_alpha=cfg.strongsort.ema_alpha 
)

### 2.4. OC-SORT [27 Mar 2022]
#### Observation-Centric SORT: Rethinking SORT for Robust Multi-Object Tracking
#### https://arxiv.org/abs/2203.14360

In [ ]:
from trackers import OCSORT

In [ ]:
tracker_config = os.path.join(os.getcwd(), "trackers", "ocsort", "configs", "ocsort.yaml")

cfg = YamlParser()
cfg.merge_from_file(tracker_config)

tracker = OCSORT(
    det_thresh=cfg.ocsort.det_thresh,
    max_age=cfg.ocsort.max_age,
    min_hits=cfg.ocsort.min_hits,
    iou_threshold=cfg.ocsort.iou_thresh,
    delta_t=cfg.ocsort.delta_t,
    asso_func=cfg.ocsort.asso_func,
    inertia=cfg.ocsort.inertia,
    use_byte=cfg.ocsort.use_byte,
)

### 2.5. ByteTrack [7 Apr 2022]
#### Multi-Object Tracking by Associating Every Detection Box
#### https://paperswithcode.com/paper/bytetrack-multi-object-tracking-by-1

In [ ]:
from trackers import BYTETrack

In [ ]:
tracker_config = os.path.join(os.getcwd(), "trackers", "bytetrack", "configs", "bytetrack.yaml")

cfg = YamlParser()
cfg.merge_from_file(tracker_config)

tracker = BYTETrack(
    track_thresh=cfg.bytetrack.track_thresh,
    match_thresh=cfg.bytetrack.match_thresh,
    track_buffer=cfg.bytetrack.track_buffer,
    frame_rate=cfg.bytetrack.frame_rate
)

### 2.6. BotSORT [7 Jul 2022]
#### Robust Associations Multi-Pedestrian Tracking
#### https://paperswithcode.com/paper/bot-sort-robust-associations-multi-pedestrian

In [ ]:
from trackers import BoTSORT

In [ ]:
reid_weights = os.path.join(os.getcwd(), "trackers", "botsort", "deep", "checkpoint", "osnet_x0_25_msmt17.pt")
tracker_config = os.path.join(os.getcwd(), "trackers", "botsort", "configs", "botsort.yaml")

cfg = YamlParser()
cfg.merge_from_file(tracker_config)

tracker = BoTSORT(
    model_weights=reid_weights,
    device=device,
    fp16=True,
    track_high_thresh=cfg.botsort.track_high_thresh,
    new_track_thresh=cfg.botsort.new_track_thresh,
    track_buffer=cfg.botsort.track_buffer,
    match_thresh=cfg.botsort.match_thresh,
    proximity_thresh=cfg.botsort.proximity_thresh,
    appearance_thresh=cfg.botsort.appearance_thresh,
    cmc_method=cfg.botsort.cmc_method,
    frame_rate=cfg.botsort.frame_rate,
    lambda_=cfg.botsort.lambda_
)

### 2.7. Deep OC-SORT [23 Feb 2023]
#### Deep OC-SORT: Multi-Pedestrian Tracking by Adaptive Re-Identification (DeepSORT + OCSORT)
#### https://arxiv.org/abs/2302.11813

In [ ]:
from trackers import DeepOCSORT

In [ ]:
reid_weights = os.path.join(os.getcwd(), "trackers", "deepocsort", "deep", "checkpoint", "osnet_x0_25_msmt17.pt")
tracker_config = os.path.join(os.getcwd(), "trackers", "deepocsort", "configs", "deepocsort.yaml")

cfg = YamlParser()
cfg.merge_from_file(tracker_config)

tracker = DeepOCSORT(
    model_weights=reid_weights,
    device=device,
    fp16=True,
    det_thresh=cfg.deepocsort.det_thresh,
    max_age=cfg.deepocsort.max_age,
    min_hits=cfg.deepocsort.min_hits,
    iou_threshold=cfg.deepocsort.iou_thresh,
    delta_t=cfg.deepocsort.delta_t,
    asso_func=cfg.deepocsort.asso_func,
    inertia=cfg.deepocsort.inertia,
)

# ----------------------------------------------------------------------------

## 3. Run Tracking

In [ ]:
tracking_runner = TrackingRunner(detector, tracker)

tracking_runner.run( </br>
&emsp;source=source, &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; # 0 for webcam; otherwise path to image/video/directory </br>
&emsp;output=output, &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; # optional; default_value=None. A output path to save an image or a video </br>
&emsp;display=False, &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; # optional; default_value = False. Displays output in a window </br>
&emsp;keyboard_interrupt_key="q" &emsp; # optional; default_value = "q". A keyboard button to stop/quit running detection </br>
)

### 3.1. Run on sequence of images

In [ ]:
source = os.path.join(os.getcwd(), "data", "inputs", "sample_sequence")
save_output = os.path.join(os.getcwd(), "data", "outputs", "tracking", "track_output_sequence.avi")
tracking_runner.run(source, save_output)

### 3.2. Run on video

In [ ]:
source = os.path.join(os.getcwd(), "data", "inputs", "sample_video.mp4")
save_output = os.path.join(os.getcwd(), "data", "outputs", "tracking", "track_output_video.avi")
tracking_runner.run(source, save_output)

### 3.3. Run on webcam

In [ ]:
# source = 0
# save_output = os.path.join(os.getcwd(), "data", "outputs", "tracking", "track_output_webcam.avi")

# tracking_runner.run(source, save_output)